# Western States Weather Weighted Average Pace Prediction

Weather is a key factor into the performance of runners, especially for ultra marathons. It
seems that in extreme heat, the performance of athletes decreases. This would also be relative
to ultra marathons. The mean yearly average pace was compared to the mean yearly average
temperature and, using linear regression, found that as the degrees Celsius temperature rises by 1
degree, there is a 0.1 rise in minutes per kilometre pace for the UTMB and Western States. Each
race was imagined to be ran at 22 degrees Celsius. Therefore, for 2016, where the temperature
was 30 degrees Celsius, (0.1 * 8) was subtracted from every average pace of every runner for
that year. This process was repeated for each year, multiplying 0.1 by the difference between the
temperatures. The prediction model was tested using these new weather weighted times.


In [310]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [311]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [312]:
# Read in Split Timings
df = pd.read_csv('../../data/WS_1km_splits_full.csv', encoding = "ISO-8859-1")

In [313]:
df = df[df['Year'] != 2010]

df = df[df['Year'] != 2012]

In [314]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'bib', 'rank', 'nationality'], axis = 1)

In [315]:
imp_cats = df['category']
imp_year = df['Year']
imp_name = df['name_mask']
imp_gen = df['gender']
imp_comb = [imp_cats, imp_gen, imp_name, imp_year]
imp_comb = pd.concat(imp_comb, axis=1)

In [316]:
df.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
0,2018,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),28.0,6.265060,NaN,5.393258,NaN,4.347826,NaN,NaN,5.894737,NaN,4.545455,4.464286,4.705882,6.805556,5.243902,6.888889,5.247525,4.666667,NaN,4.75,6.216216,NaN,19.655172,5.862069,NaN,5.280899,NaN,6.729885,NaN,NaN,11.083333,4.928571,870.066667,6.448759
1,2018,343c649718c8ed904090cbcb05ca97b1,Male,Senior Men (23-39),32.0,6.204819,NaN,5.842697,NaN,4.782609,NaN,NaN,6.105263,NaN,4.696970,4.821429,4.705882,6.944444,4.878049,8.444444,6.138614,5.833333,NaN,5.50,6.486486,NaN,23.103448,6.321839,NaN,5.842697,NaN,7.931034,NaN,NaN,14.838235,6.873016,954.883333,7.314765
2,2018,275e1385c00219e79ea54c8d6381fc07,Male,Senior Men (23-39),33.0,6.325301,NaN,6.067416,NaN,4.927536,NaN,NaN,6.631579,NaN,5.454545,NaN,8.000000,7.361111,5.000000,8.666667,5.742574,5.333333,NaN,5.75,7.027027,NaN,22.758621,6.206897,NaN,5.842697,NaN,7.586207,NaN,NaN,14.632353,6.777778,968.983333,7.689034
3,2018,9a22e0603cd00f47e015963c32d6f964,Male,Senior Men (23-39),37.0,6.385542,NaN,6.179775,NaN,4.927536,NaN,NaN,6.736842,NaN,5.000000,NaN,8.000000,7.638889,5.121951,9.111111,6.336634,5.333333,NaN,5.75,7.027027,NaN,23.793103,6.321839,NaN,6.067416,NaN,7.758621,NaN,NaN,13.906863,6.309524,983.533333,7.774000
4,2018,0d274b89ac57a27941405195d37bf2c3,Male,Senior Men (23-39),31.0,6.566265,NaN,6.292135,NaN,5.000000,NaN,NaN,6.947368,NaN,5.000000,5.178571,5.176471,8.750000,5.000000,10.666667,6.790429,5.569444,NaN,6.25,7.837838,NaN,24.827586,6.436782,NaN,5.617978,NaN,7.931034,NaN,NaN,12.455882,4.301587,1014.383333,7.629802


##  Weighting pace according to weather

1 degree rise in temperature causes a 0.08 rise in average pace. Weighted times as if all races were ran in 22 degrees.

In [317]:
df18 = df[df.Year == 2018]

In [318]:
df18 = df18.loc[:, 'Start to Lyon Ridge':'AVG_Pace'] - (0.1 * 5)

In [319]:
df17 = df[df.Year == 2017]

In [320]:
df17 = df17.loc[:, 'Start to Lyon Ridge':'AVG_Pace'] - 0.1

In [321]:
df16 = df[df.Year == 2016]

In [322]:
df16 = df16.loc[:, 'Start to Lyon Ridge':'AVG_Pace'] 

In [323]:
df15 = df[df.Year == 2015]

In [324]:
df15 = df15.loc[:, 'Start to Lyon Ridge':'AVG_Pace'] 

In [325]:
df14 = df[df.Year == 2014]

In [326]:
df14 = df14.loc[:, 'Start to Lyon Ridge':'AVG_Pace'] + (0.1 * 2)

In [327]:
df13 = df[df.Year == 2013]

In [328]:
df13 = df13.loc[:, 'Start to Lyon Ridge':'AVG_Pace'] - (0.1 * 5)

In [329]:
#year_frames = [df18, df17, df16, df15, df14, df13]
year_frames = [df17, df16, df15, df14, df13]
df = pd.concat(year_frames)

In [330]:
df = df.join(imp_comb)

In [331]:
df = df[['Year',              
'name_mask'           ,
              'gender',
         'category',
'time'             ,
'Start to Lyon Ridge'            ,
'Start to Talbot Creek'          ,
'Lyon Ridge to Red Star Ridge'          ,
'Talbot Creek to Poppy Trailhead'            ,
'Red Star Ridge to Duncan Canyon'          ,
'Poppy Trailhead to Duncan Canyon'          ,
'Duncan Canyon to Mosquito Ridge'          ,
'Duncan Canyon to Robinson Flat'            ,
'Mosquito Ridge to Millers Defeat'          ,
'Robinson Flat to Millers Defeat'          ,
'Millers Defeat to Dusty Corners'          ,
'Dusty Corners to Last Chance'          ,
'Last Chance to Devils Thumb'            ,
'Devils Thumb to El Dorado Creek'          ,
'El Dorado Creek to Michigan Bluff'          ,
'Michigan Bluff to Foresthill'          ,
'Foresthill to Dardanelles'         ,
'Foresthill to Peachstone'        ,
'Dardanelles to Peachstone'         ,
'Peachstone to Fords Bar'         ,
'Peachstone to Rucky Chucky'        ,
'Rucky Chucky to Green Gate'          ,
'Green Gate to Auburn Lake Trails'          ,
'Auburn Lake Trails to Browns Bar'        ,
'Auburn Lake Trails to Quarry Road'         ,
'Browns Bar to Highway 49'         ,
'Quarry Road to Pointed Rocks'          ,
'Highway 49 to No Hands Bridge'        ,
'Pointed Rocks to No Hands Bridge'         ,
'No Hands Bridge to Robie Point'        ,
'Robie Point to Finish',
        'AVG_Pace']]

In [332]:
df.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
111,2017,10c5f67365f4b5d4974e26b9d8494f25,Male,Senior Men (23-39),979.516667,6.658032,NaN,5.946816,NaN,5.044928,NaN,NaN,6.215789,NaN,4.596970,4.721429,4.252941,7.677778,4.046341,11.455556,6.134323,NaN,5.902381,NaN,NaN,5.455556,9.210345,6.221839,NaN,5.630337,NaN,6.968966,NaN,NaN,13.429412,6.384127,6.629151
112,2017,196d4c89233fe69a5ff5b18e68440d8c,Male,Senior Men (23-39),1008.283333,6.748394,NaN,6.115356,NaN,5.552174,NaN,NaN,6.531579,NaN,4.900000,5.078571,4.488235,7.677778,5.387805,8.122222,5.543564,NaN,5.971429,NaN,NaN,6.224786,8.175862,6.911494,NaN,5.967416,NaN,7.831034,NaN,NaN,13.429412,5.796825,6.655470
113,2017,275e1385c00219e79ea54c8d6381fc07,Male,Senior Men (23-39),1012.850000,6.804618,NaN,6.572285,NaN,5.334783,NaN,NaN,6.742105,NaN,4.596970,5.435714,4.488235,7.955556,5.143902,9.677778,6.018812,NaN,5.842857,NaN,NaN,5.882906,7.831034,6.221839,NaN,6.079775,NaN,7.486207,NaN,NaN,13.723529,6.542857,6.756935
114,2017,9b61a10caab917ad2d0e7b1621007ac5,Male,Senior Men (23-39),1056.083333,6.760442,NaN,6.092884,NaN,5.479710,NaN,NaN,6.636842,NaN,4.900000,5.078571,4.488235,8.094444,5.509756,10.788889,6.866997,NaN,6.230952,NaN,NaN,5.968376,9.210345,6.911494,NaN,6.866292,NaN,8.003448,NaN,NaN,14.900000,6.177778,7.103445
115,2017,b06c1c4b8fe7d9f6f7cd49e3cba22198,Male,Senior Men (23-39),1057.083333,6.778514,NaN,6.620974,NaN,5.334783,NaN,NaN,6.952632,NaN,5.354545,5.614286,5.076471,8.372222,5.631707,9.011111,6.386799,NaN,5.720238,NaN,NaN,NaN,31.624138,7.026437,NaN,7.091011,NaN,8.865517,NaN,NaN,15.194118,5.225397,8.437828


In [333]:
df.groupby('category').count()

,Year,name_mask,gender,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),588,588,588,448,549,0,574,0,562,0,0,574,0,536,521,527,541,426,525,509,0,478,0,0,466,464,462,357,85,367,86,99,0,436,447,584
Masters Men 2 (50-59),268,268,268,177,250,0,258,0,255,0,0,261,0,234,232,226,238,182,227,218,0,202,0,0,192,188,183,150,29,148,29,35,0,174,175,266
Masters Men 3 (60-69),56,56,56,32,51,0,55,0,54,0,0,53,0,50,48,50,50,35,46,43,0,41,0,0,37,37,37,32,4,30,4,8,0,33,32,56
Masters Men 4 (70-79),5,5,5,0,4,0,5,0,4,0,0,4,0,4,4,3,4,2,2,2,0,2,0,0,2,2,2,1,0,1,0,0,0,1,0,5
Masters Women 1 (40-49),145,145,145,108,126,0,136,0,137,0,0,136,0,124,122,124,129,97,126,121,0,118,0,0,111,113,112,94,13,94,13,22,0,104,106,143
Masters Women 2 (50-59),71,71,71,37,62,0,69,0,65,0,0,61,0,55,51,52,52,35,48,44,0,43,0,0,43,41,40,36,5,35,4,6,0,37,37,71
Masters Women 3 (60-69),5,5,5,2,4,0,5,0,5,0,0,5,0,3,2,3,3,3,3,3,0,3,0,0,3,2,2,1,1,1,1,0,0,2,2,5
Masters Women 4 (70-79),3,3,3,1,2,0,3,0,3,0,0,2,0,2,2,2,2,1,2,2,0,2,0,0,2,1,1,1,0,1,0,0,0,1,1,3
Senior Men (23-39),529,529,529,418,500,0,515,0,507,0,0,521,0,497,491,491,491,397,484,465,0,441,0,0,426,420,419,341,77,341,76,94,0,407,415,526


In [334]:
#df = df.drop([7767])
#df = df.drop([10182])

In [335]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1994,2015,001e405237215c01ebb54168bc0d93ec,Male,Masters Men 2 (50-59),1669.033333,7.951807,NaN,8.539326,NaN,7.028986,NaN,NaN,11.684211,NaN,10.909091,8.214286,8.117647,13.472222,9.756098,14.666667,11.683168,NaN,10.071429,NaN,NaN,11.880342,19.310345,10.919540,10.649351,NaN,14.259259,NaN,NaN,NaN,28.299020,8.960317,11.914374
1436,2014,003e885586c8ce6413604f5cc8ead272,Male,Masters Men 1 (40-49),1737.666667,8.332530,NaN,8.851685,NaN,7.591304,NaN,NaN,10.831579,NaN,12.321212,10.200000,8.435294,12.422222,11.419512,13.977778,11.388119,NaN,11.557143,NaN,NaN,NaN,76.751724,11.924138,11.368831,NaN,12.607407,NaN,8.578378,NaN,13.317647,9.184127,14.266349
124,2017,00dd2d2002b5d45f417ae37cb4bc29e3,Male,Senior Men (23-39),1262.800000,6.942169,NaN,6.540449,NaN,5.914493,NaN,NaN,6.847368,NaN,5.809091,5.614286,5.076471,9.900000,6.363415,11.455556,7.532013,NaN,7.322619,NaN,NaN,9.814530,9.210345,8.060920,NaN,10.686517,NaN,10.762069,NaN,NaN,20.194118,10.804762,8.676378
1134,2016,0104b6fc44ae66e38da37734f708ab1f,Male,Masters Men 1 (40-49),1369.816667,7.464859,NaN,7.554307,NaN,6.221014,NaN,NaN,9.473684,NaN,9.696970,6.785714,6.352941,10.972222,NaN,29.333333,8.480198,NaN,8.453571,NaN,NaN,7.863248,13.448276,8.620690,8.831169,NaN,10.555556,NaN,NaN,NaN,NaN,48.007937,12.242099
1018,2017,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.484270,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [336]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [337]:
df.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1018,2017,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
1378,2014,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974


# Merge

In [338]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [339]:
pb.head()

,name_mask,gender,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1018,0106a626dcce58322c9f90dfcd777b9c,Male,1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1378,017b3bcb28ddc8e5336d606e57611f84,Male,1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
2043,0530f3b9ddc07dcdff4e1363db2b13ad,Male,1517.750000,8.754217,NaN,8.851685,NaN,7.446377,NaN,NaN,10.515789,NaN,10.503030,9.664286,NaN,19.644444,8.492683,13.088889,9.308911,NaN,8.842857,NaN,NaN,NaN,49.165517,10.544828,12.148052,NaN,13.903704,NaN,NaN,NaN,26.356863,6.684127,13.759780
2052,066dd61e63158704bcb0e6ee2bd5991e,Male,1664.783333,8.151807,NaN,8.402247,NaN,7.591304,NaN,NaN,10.621053,NaN,11.109091,7.878571,NaN,22.561111,10.931707,14.200000,10.794059,NaN,10.985714,NaN,NaN,11.396581,15.717241,11.464368,12.927273,NaN,14.088889,NaN,9.929730,NaN,10.827451,8.985714,11.503364
2481,0a43ae8070c7d12e36c34816c6278951,Female,1267.666667,7.902610,NaN,7.764045,NaN,6.283816,NaN,NaN,8.842105,NaN,7.878788,6.428571,5.764706,10.000000,NaN,24.000000,7.613861,NaN,7.435714,NaN,NaN,7.692308,11.724138,8.160920,7.662338,NaN,9.814815,NaN,NaN,NaN,22.352941,7.460317,9.710111


In [340]:
npb.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974
1996,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286
2510,2015,0a43ae8070c7d12e36c34816c6278951,Female,Senior Women (23-39),1307.416667,8.149598,NaN,8.282772,NaN,6.449275,NaN,NaN,10.315789,NaN,7.424242,6.785714,6.000000,10.277778,7.195122,12.444444,7.722772,NaN,7.285714,NaN,NaN,8.205128,11.724138,8.620690,8.051948,NaN,9.444444,NaN,NaN,NaN,21.004902,7.142857,9.080386


In [341]:
npb[npb.AVG_Pace.isnull()]

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace


In [342]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [343]:
#merge = merge.groupby('name_mask').head(1)

In [344]:
first_df = len(merge)

In [345]:
first_df

153

In [346]:
merge.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720,Male,1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579,Male,1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974,Male,1517.750000,8.754217,NaN,8.851685,NaN,7.446377,NaN,NaN,10.515789,NaN,10.503030,9.664286,NaN,19.644444,8.492683,13.088889,9.308911,NaN,8.842857,NaN,NaN,NaN,49.165517,10.544828,12.148052,NaN,13.903704,NaN,NaN,NaN,26.356863,6.684127,13.759780
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286,

In [347]:
merge.groupby('category').count()

,Year,name_mask,gender,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),51,51,51,51,48,0,49,0,51,0,0,51,0,49,45,50,50,39,51,51,0,48,0,0,50,50,51,37,13,38,13,5,0,49,51,51,51,51,48,0,51,0,43,0,0,51,0,51,47,50,51,45,51,51,0,50,0,0,47,50,51,46,5,46,5,24,0,51,51,51
Masters Men 2 (50-59),22,22,22,22,22,0,21,0,21,0,0,22,0,22,21,21,22,16,22,22,0,22,0,0,21,22,22,15,7,15,7,3,0,21,22,22,22,22,21,0,22,0,22,0,0,22,0,22,22,19,22,20,22,22,0,22,0,0,21,21,22,19,1,21,1,7,0,22,22,22
Masters Men 3 (60-69),5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,2,5,5,0,5,0,0,5,5,5,4,1,4,1,1,0,5,5,5,5,5,5,0,5,0,4,0,0,5,0,5,5,5,5,5,5,5,0,5,0,0,4,5,5,5,0,5,0,1,0,5,5,5
Masters Women 1 (40-49),12,12,12,12,11,0,12,0,11,0,0,12,0,12,11,12,12,10,12,12,0,12,0,0,12,12,12,11,1,11,1,2,0,12,12,12,12,12,12,0,11,0,12,0,0,12,0,12,12,12,12,12,12,12,0,12,0,0,12,12,12,10,2,9,2,3,0,12,12,12
Masters Women 2 (50-59),5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,4,5,5,0,5,0,0,5,5,5,4,1,4,1,1,0,5,5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,4,5,5,0,5,0,0,5,5,5,5,0,5,0,0,0,5,5,5
Senior Men (23-39),44,44,44,44,43,0,43,0,44,0,0,44,0,42,43,41,43,39,44,44,0,44,0,0,43,43,44,32,12,32,12,9,0,44,44,44,44,44,44,0,44,0,43,0,0,44,0,44,44,41,44,34,44,44,0,42,0,0,44,44,44,38,6,38,6,15,0,42,44,44
Senior Women (23-39),14,14,14,14,13,0,14,0,12,0,0,14,0,14,14,14,14,13,14,14,0,13,0,0,14,14,14,11,3,11,3,4,0,14,14,14,14,14,13,0,14,0,14,0,0,14,0,14,14,14,14,6,14,14,0,14,0,0,14,14,14,14,0,13,0,1,0,14,14,14


In [348]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1018,2017,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
1378,2014,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [349]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [350]:
df.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1018,2017,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
1378,2014,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974


# Merge

In [351]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [352]:
pb.head()

,name_mask,gender,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1018,0106a626dcce58322c9f90dfcd777b9c,Male,1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1378,017b3bcb28ddc8e5336d606e57611f84,Male,1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
2043,0530f3b9ddc07dcdff4e1363db2b13ad,Male,1517.750000,8.754217,NaN,8.851685,NaN,7.446377,NaN,NaN,10.515789,NaN,10.503030,9.664286,NaN,19.644444,8.492683,13.088889,9.308911,NaN,8.842857,NaN,NaN,NaN,49.165517,10.544828,12.148052,NaN,13.903704,NaN,NaN,NaN,26.356863,6.684127,13.759780
2052,066dd61e63158704bcb0e6ee2bd5991e,Male,1664.783333,8.151807,NaN,8.402247,NaN,7.591304,NaN,NaN,10.621053,NaN,11.109091,7.878571,NaN,22.561111,10.931707,14.200000,10.794059,NaN,10.985714,NaN,NaN,11.396581,15.717241,11.464368,12.927273,NaN,14.088889,NaN,9.929730,NaN,10.827451,8.985714,11.503364
2481,0a43ae8070c7d12e36c34816c6278951,Female,1267.666667,7.902610,NaN,7.764045,NaN,6.283816,NaN,NaN,8.842105,NaN,7.878788,6.428571,5.764706,10.000000,NaN,24.000000,7.613861,NaN,7.435714,NaN,NaN,7.692308,11.724138,8.160920,7.662338,NaN,9.814815,NaN,NaN,NaN,22.352941,7.460317,9.710111


In [353]:
npb.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974
1996,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286
2510,2015,0a43ae8070c7d12e36c34816c6278951,Female,Senior Women (23-39),1307.416667,8.149598,NaN,8.282772,NaN,6.449275,NaN,NaN,10.315789,NaN,7.424242,6.785714,6.000000,10.277778,7.195122,12.444444,7.722772,NaN,7.285714,NaN,NaN,8.205128,11.724138,8.620690,8.051948,NaN,9.444444,NaN,NaN,NaN,21.004902,7.142857,9.080386


In [354]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [355]:
merge.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720,Male,1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579,Male,1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974,Male,1517.750000,8.754217,NaN,8.851685,NaN,7.446377,NaN,NaN,10.515789,NaN,10.503030,9.664286,NaN,19.644444,8.492683,13.088889,9.308911,NaN,8.842857,NaN,NaN,NaN,49.165517,10.544828,12.148052,NaN,13.903704,NaN,NaN,NaN,26.356863,6.684127,13.759780
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286,

In [356]:
merge.groupby('category').count()

,Year,name_mask,gender,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),51,51,51,51,48,0,49,0,51,0,0,51,0,49,45,50,50,39,51,51,0,48,0,0,50,50,51,37,13,38,13,5,0,49,51,51,51,51,48,0,51,0,43,0,0,51,0,51,47,50,51,45,51,51,0,50,0,0,47,50,51,46,5,46,5,24,0,51,51,51
Masters Men 2 (50-59),22,22,22,22,22,0,21,0,21,0,0,22,0,22,21,21,22,16,22,22,0,22,0,0,21,22,22,15,7,15,7,3,0,21,22,22,22,22,21,0,22,0,22,0,0,22,0,22,22,19,22,20,22,22,0,22,0,0,21,21,22,19,1,21,1,7,0,22,22,22
Masters Men 3 (60-69),5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,2,5,5,0,5,0,0,5,5,5,4,1,4,1,1,0,5,5,5,5,5,5,0,5,0,4,0,0,5,0,5,5,5,5,5,5,5,0,5,0,0,4,5,5,5,0,5,0,1,0,5,5,5
Masters Women 1 (40-49),12,12,12,12,11,0,12,0,11,0,0,12,0,12,11,12,12,10,12,12,0,12,0,0,12,12,12,11,1,11,1,2,0,12,12,12,12,12,12,0,11,0,12,0,0,12,0,12,12,12,12,12,12,12,0,12,0,0,12,12,12,10,2,9,2,3,0,12,12,12
Masters Women 2 (50-59),5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,4,5,5,0,5,0,0,5,5,5,4,1,4,1,1,0,5,5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,4,5,5,0,5,0,0,5,5,5,5,0,5,0,0,0,5,5,5
Senior Men (23-39),44,44,44,44,43,0,43,0,44,0,0,44,0,42,43,41,43,39,44,44,0,44,0,0,43,43,44,32,12,32,12,9,0,44,44,44,44,44,44,0,44,0,43,0,0,44,0,44,44,41,44,34,44,44,0,42,0,0,44,44,44,38,6,38,6,15,0,42,44,44
Senior Women (23-39),14,14,14,14,13,0,14,0,12,0,0,14,0,14,14,14,14,13,14,14,0,13,0,0,14,14,14,11,3,11,3,4,0,14,14,14,14,14,13,0,14,0,14,0,0,14,0,14,14,14,14,6,14,14,0,14,0,0,14,14,14,14,0,13,0,1,0,14,14,14


In [357]:
merge = merge[merge.name_mask != 'f74186d56d674bb6eb38a44bd8a579a3']
merge = merge[merge.name_mask != '25ad5e3dae25b9ac468d5590a15b9b87']

In [358]:
merge = merge.reset_index(drop=True)

In [359]:
merge.iloc[20]

Year                                                                2016
name_mask                               272e9cf1cf51b64d07cf8e7dc845d0e7
gender                                                            Female
category                                            Senior Women (23-39)
time                                                             1311.43
Start to Lyon Ridge                                              7.18173
Start to Talbot Creek                                                NaN
Lyon Ridge to Red Star Ridge                                     7.29963
Talbot Creek to Poppy Trailhead                                      NaN
Red Star Ridge to Duncan Canyon                                  5.85628
Poppy Trailhead to Duncan Canyon                                     NaN
Duncan Canyon to Mosquito Ridge                                      NaN
Duncan Canyon to Robinson Flat                                   8.94737
Mosquito Ridge to Millers Defeat                   

In [360]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [361]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [362]:
merge.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720,Male,1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579,Male,1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974,Male,1517.750000,8.754217,NaN,8.851685,NaN,7.446377,NaN,NaN,10.515789,NaN,10.503030,9.664286,NaN,19.644444,8.492683,13.088889,9.308911,NaN,8.842857,NaN,NaN,NaN,49.165517,10.544828,12.148052,NaN,13.903704,NaN,NaN,NaN,26.356863,6.684127,13.759780
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286,

In [363]:
pb.head()

,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,Male,1516.300000,8.618876,NaN,8.244569,NaN,6.494203,NaN,NaN,9.478947,NaN,7.627273,7.042857,6.605882,11.983333,8.436585,13.677778,8.779538,NaN,8.708333,NaN,NaN,9.301709,14.727586,9.900000,NaN,12.48427,NaN,13.865517,NaN,NaN,24.017647,9.614286,10.505747
1,Male,1397.466667,8.513253,NaN,8.514607,NaN,7.518841,NaN,NaN,10.094737,NaN,9.442424,7.342857,6.082353,12.422222,7.151220,13.977778,9.011881,NaN,8.414286,NaN,NaN,8.148718,12.958621,7.901149,7.472727,NaN,11.311111,NaN,7.767568,NaN,12.523529,8.469841,9.251986
2,Male,1517.750000,8.754217,NaN,8.851685,NaN,7.446377,NaN,NaN,10.515789,NaN,10.503030,9.664286,NaN,19.644444,8.492683,13.088889,9.308911,NaN,8.842857,NaN,NaN,NaN,49.165517,10.544828,12.148052,NaN,13.903704,NaN,NaN,NaN,26.356863,6.684127,13.759780
3,Male,1664.783333,8.151807,NaN,8.402247,NaN,7.591304,NaN,NaN,10.621053,NaN,11.109091,7.878571,NaN,22.561111,10.931707,14.200000,10.794059,NaN,10.985714,NaN,NaN,11.396581,15.717241,11.464368,12.927273,NaN,14.088889,NaN,9.929730,NaN,10.827451,8.985714,11.503364
4,Female,1267.666667,7.902610,NaN,7.764045,NaN,6.283816,NaN,NaN,8.842105,NaN,7.878788,6.428571,5.764706,10.000000,NaN,24.000000,7.613861,NaN,7.435714,NaN,NaN,7.692308,11.724138,8.160920,7.662338,NaN,9.814815,NaN,NaN,NaN,22.352941,7.460317,9.710111


In [364]:
pb.to_csv('../../data/wea_ws1_pb_times.csv')

In [365]:
npb.head()

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286
4,2015,0a43ae8070c7d12e36c34816c6278951,Female,Senior Women (23-39),1307.416667,8.149598,NaN,8.282772,NaN,6.449275,NaN,NaN,10.315789,NaN,7.424242,6.785714,6.000000,10.277778,7.195122,12.444444,7.722772,NaN,7.285714,NaN,NaN,8.205128,11.724138,8.620690,8.051948,NaN,9.444444,NaN,NaN,NaN,21.004902,7.142857,9.080386


# Define Functions

In [366]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_gender = npb[npb.gender == query.gender]
    similar_times = same_gender[(same_gender.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_gender.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(0)

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),1668.800000,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,11.804720
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),1623.266667,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,12.242579
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),1573.350000,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,10.913974
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),1681.783333,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,11.844286
15,2014,19a2daf8b8592e01f851c4b51319899c,Male,Masters Men 2 (50-59),1654.766667,8.212048,NaN,8.626966,NaN,7.446377,NaN,NaN,9.778947,NaN,9.593939,8.057143,7.376471,12.144444,8.370732,13.755556,11.190099,NaN,10.842857,NaN,NaN,15.071795,20.889655,11.694253,10.979221,NaN,13.348148,NaN,8.037838,NaN,14.141176,10.279365,10.991852
16,2015,1cfacfe60837958f1be7a2b13aaac0b5,Male,Senior Men (23-39),1580.883333,9.295181,NaN,8.842697,NaN,6.594203,NaN,NaN,10.947368,NaN,10.000000,8.035714,8.235294,13.472222,9.390244,14.222222,10.594059,NaN,8.428571,NaN,NaN,10.683761,16.896552,9.080460,9.480519,NaN,12.962963,NaN,NaN,NaN,27.196078,9.722222,11.267386
17,2016,1ea16509e1c34400de4a0af729413bb8,Male,Masters Men 1 (40-49),1653.366667,8.086345,NaN,7.975655,NaN,6.868357,NaN,NaN,10.947368,NaN,10.606061,8.214286,8.117647,12.361111,NaN,30.666667,10.463696,NaN,9.665476,NaN,NaN,13.931624,14.482759,12.528736,11.298701,NaN,13.333333,NaN,NaN,NaN,31.176471,8.269841,12.721896
24,2014,35d21750998dfcc6976121ddc205725f,Male,Senior Men (23-39),1582.700000,9.296386,NaN,9.188764,NaN,7.953623,NaN,NaN,10.621053,NaN,9.896970,7.164286,NaN,21.311111,9.346341,13.755556,10.497030,NaN,8.200000,NaN,NaN,10.883761,15.027586,9.050575,11.888312,NaN,13.348148,NaN,10.200000,NaN,11.940196,9.049206,10.979942
26,2013,3f2963c8306fa48b5550122a05d185b5,Male,Masters Men 1 (40-49),1790.583333,9.560241,NaN,11.522472,NaN,8.195652,NaN,NaN,10.868421,NaN,11.166667,8.964286,8.676471,5.611111,17.304878,13.722222,10.391089,NaN,10.500000,NaN,NaN,11.636752,15.362069,12.718391,9.889610,NaN,11.166667,NaN,NaN,NaN,27.735294,9.539683,11.817472
27,2015,420266030cd34eb1924b8a01967ce661,Male,Senior Men (23-39),1633.683333,8.222892,NaN,7.696629,NaN,6.304348,NaN,NaN,8.736842,NaN,8.484848,6.785714,6.470588,12.777778,8.536585,15.333333

In [367]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(0)

69     0.997827
16     0.996600
95     0.996336
145    0.994607
148    0.994401
121    0.994208
134    0.993838
107    0.992714
2      0.990949
78     0.990825
dtype: float64

In [368]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(30)

40     13.327349
65     13.359113
60     13.560507
20     18.602472
150    19.150254
62     20.351766
11     20.428543
dtype: float64

In [369]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(30)

60     0.146294
40     0.146245
65     0.145693
150    0.143366
20     0.140804
62     0.139126
11     0.138472
dtype: float64

In [370]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(66)

141    0.107164
8      0.104231
21     0.103661
113    0.103326
138    0.101148
4      0.099163
7      0.097022
126    0.095760
136    0.095511
5      0.093014
dtype: float64

In [371]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb,weights,weighted_pace
69,2015,758ede5c3802b99099cee83399a2c5aa,Male,Masters Men 2 (50-59),1653.283333,8.146586,NaN,8.850187,NaN,6.666667,NaN,NaN,10.736842,NaN,9.696970,8.571429,7.647059,13.888889,8.292683,14.666667,10.594059,NaN,10.142857,NaN,NaN,11.709402,16.206897,11.839080,11.948052,NaN,14.629630,NaN,NaN,NaN,30.294118,11.563492,11.899556,Male,1535.016667,7.572289,NaN,7.589888,NaN,6.021739,NaN,NaN,9.078947,NaN,8.439394,7.000000,5.617647,10.888889,6.939024,12.611111,8.509901,NaN,7.142857,NaN,NaN,8.730769,15.017241,12.028736,NaN,NaN,33.018519,NaN,NaN,NaN,35.970588,13.079365,11.958717,0.100362,1.200198
16,2015,1cfacfe60837958f1be7a2b13aaac0b5,Male,Senior Men (23-39),1580.883333,9.295181,NaN,8.842697,NaN,6.594203,NaN,NaN,10.947368,NaN,10.000000,8.035714,8.235294,13.472222,9.390244,14.222222,10.594059,NaN,8.428571,NaN,NaN,10.683761,16.896552,9.080460,9.480519,NaN,12.962963,NaN,NaN,NaN,27.196078,9.722222,11.267386,Male,1495.183333,9.029518,NaN,7.478652,NaN,6.856522,NaN,NaN,10.847368,NaN,8.233333,6.685714,6.958824,10.455556,7.460976,14.566667,12.114521,NaN,9.516667,NaN,NaN,9.900000,12.658621,8.750575,NaN,9.338202,NaN,10.417241,NaN,NaN,18.723529,10.034921,10.001442,0.100238,1.002528
95,2015,b073a967413c1f902bdbd5efb9885527,Male,Masters Men 1 (40-49),1597.733333,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,11.130149,Male,1360.266667,8.812651,NaN,8.782022,NaN,7.001449,NaN,NaN,9.794737,NaN,7.475758,7.221429,6.723529,11.011111,7.339024,12.344444,7.749835,NaN,8.022619,NaN,NaN,7.677778,10.934483,8.635632,NaN,8.439326,NaN,9.555172,NaN,NaN,16.370588,7.217460,9.005739,0.100212,0.902481
145,2017,fac9159d6fc8daca441581e857c3fae6,Male,Masters Men 1 (40-49),1728.466667,10.800602,NaN,9.793258,NaN,8.088406,NaN,NaN,11.163158,NaN,8.536364,8.292857,7.782353,13.094444,9.656098,14.566667,10.386799,NaN,10.834524,NaN,NaN,11.694872,14

In [372]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(0)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb,weights,weighted_pace
69,2015,758ede5c3802b99099cee83399a2c5aa,Male,Masters Men 2 (50-59),1653.283333,8.146586,NaN,8.850187,NaN,6.666667,NaN,NaN,10.736842,NaN,9.696970,8.571429,7.647059,13.888889,8.292683,14.666667,10.594059,NaN,10.142857,NaN,NaN,11.709402,16.206897,11.839080,11.948052,NaN,14.629630,NaN,NaN,NaN,30.294118,11.563492,11.899556,Male,1535.016667,7.572289,NaN,7.589888,NaN,6.021739,NaN,NaN,9.078947,NaN,8.439394,7.000000,5.617647,10.888889,6.939024,12.611111,8.509901,NaN,7.142857,NaN,NaN,8.730769,15.017241,12.028736,NaN,NaN,33.018519,NaN,NaN,NaN,35.970588,13.079365,11.958717,0.104591,1.250780
16,2015,1cfacfe60837958f1be7a2b13aaac0b5,Male,Senior Men (23-39),1580.883333,9.295181,NaN,8.842697,NaN,6.594203,NaN,NaN,10.947368,NaN,10.000000,8.035714,8.235294,13.472222,9.390244,14.222222,10.594059,NaN,8.428571,NaN,NaN,10.683761,16.896552,9.080460,9.480519,NaN,12.962963,NaN,NaN,NaN,27.196078,9.722222,11.267386,Male,1495.183333,9.029518,NaN,7.478652,NaN,6.856522,NaN,NaN,10.847368,NaN,8.233333,6.685714,6.958824,10.455556,7.460976,14.566667,12.114521,NaN,9.516667,NaN,NaN,9.900000,12.658621,8.750575,NaN,9.338202,NaN,10.417241,NaN,NaN,18.723529,10.034921,10.001442,0.102061,1.020757
145,2017,fac9159d6fc8daca441581e857c3fae6,Male,Masters Men 1 (40-49),1728.466667,10.800602,NaN,9.793258,NaN,8.088406,NaN,NaN,11.163158,NaN,8.536364,8.292857,7.782353,13.094444,9.656098,14.566667,10.386799,NaN,10.834524,NaN,NaN,11.694872,14.382759,11.509195,NaN,11.135955,NaN,14.037931,NaN,NaN,25.488235,8.741270,11.578197,Male,1717.266667,8.754217,NaN,9.301124,NaN,8.388406,NaN,NaN,12.305263,NaN,11.563636,NaN,15.023529,14.644444,9.712195,14.866667,11.586139,NaN,10.842857,NaN,NaN,10.712821,18.820690,11.579310,11.368831,NaN,12.792593,NaN,8.578378,NaN,12.552941,9.279365,11.719653,0.101044,1.184196
95,2015,b073a967413c1f902bdbd5efb9885527,Male,Masters Men 1 (40-49),1597.733333,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,1

In [373]:
def find_nearest_neighbours_pace(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    #nearest_neighbours = nearest_neighbours.filter(like='to')
    #nearest_neighbours['weights'] = weight(index)
    #nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weighted_pace'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours_pace(5)

,Year,name_mask,gender,category,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,gender_pb,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
4,2015,0a43ae8070c7d12e36c34816c6278951,Female,Senior Women (23-39),1307.416667,8.149598,NaN,8.282772,NaN,6.449275,NaN,NaN,10.315789,NaN,7.424242,6.785714,6.000000,10.277778,7.195122,12.444444,7.722772,NaN,7.285714,NaN,NaN,8.205128,11.724138,8.620690,8.051948,NaN,9.444444,NaN,NaN,NaN,21.004902,7.142857,9.080386,Female,1267.666667,7.902610,NaN,7.764045,NaN,6.283816,NaN,NaN,8.842105,NaN,7.878788,6.428571,5.764706,10.000000,NaN,24.000000,7.613861,NaN,7.435714,NaN,NaN,7.692308,11.724138,8.160920,7.662338,NaN,9.814815,NaN,NaN,NaN,22.352941,7.460317,9.710111
126,2015,da2c84850a03ed9195eb5e1ca9acb970,Female,Masters Women 1 (40-49),1321.916667,6.987952,NaN,7.640449,NaN,5.434783,NaN,NaN,9.473684,NaN,9.090909,6.607143,5.764706,10.138889,6.463415,11.777778,7.821782,NaN,8.071429,NaN,NaN,8.717949,12.413793,9.655172,9.870130,NaN,10.925926,NaN,NaN,NaN,24.656863,7.182540,9.405015,Female,1161.216667,6.307229,NaN,6.466292,NaN,5.079710,NaN,NaN,7.500000,NaN,7.378788,5.928571,4.911765,8.527778,5.963415,9.722222,6.727723,NaN,6.714286,NaN,NaN,6.252137,8.810345,7.316092,6.642857,NaN,8.388889,NaN,NaN,NaN,20.088235,6.507937,7.643909
140,2015,e8fce977f0ecd7532278e7c0ff2c35ff,Female,Masters Women 2 (50-59),1356.783333,7.590361,NaN,7.752809,NaN,6.159420,NaN,NaN,9.052632,NaN,8.030303,6.607143,6.000000,10.694444,6.951220,12.888889,8.019802,NaN,7.428571,NaN,NaN,8.205128,11.724138,9.770115,9.480519,NaN,11.296296,NaN,NaN,NaN,30.715686,9.214286,9.872724,Female,1170.333333,6.487952,NaN,6.803371,NaN,5.007246,NaN,NaN,7.184211,NaN,7.075758,5.750000,4.794118,8.805556,5.597561,11.277778,6.925743,NaN,6.571429,NaN,NaN,6.508547,9.844828,7.086207,6.642857,NaN,8.944444,NaN,NaN,NaN,19.205882,7.039683,7.765956
143,2015,f84f940684e04bbb773ae4a48626f5e6,Female,Masters Women 1 (40-49),1366.316667,7.469880,NaN,7.752809,NaN,6.086957,NaN,NaN,8.315789,NaN,8.030303,5.892857,5.529412,9.722222,6.585366,13.111111,8.415842,NaN,7.000000,NaN,NaN,8.290598,13.103448,10.574713,11.818182,NaN,14.074074,NaN,NaN,NaN,29.372549,8.309524,9.971349,Female,111

In [374]:
def pace_rec(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

pace_rec(0)

,Year,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,2015.2,1633.063333,8.780301,NaN,8.99573,NaN,7.328551,NaN,NaN,11.024737,NaN,10.172424,8.251429,7.659412,13.555,9.661707,14.53,10.095512,NaN,10.003452,NaN,NaN,10.981282,15.516207,10.631149,10.083604,11.922472,12.511188,13.77931,NaN,NaN,26.414804,8.914127,11.354317,1555.42,8.711647,NaN,8.683708,NaN,7.393451,NaN,NaN,11.781579,NaN,9.404545,7.766964,8.706536,13.047222,8.562331,15.483333,9.979538,NaN,9.366931,NaN,NaN,10.797657,18.777586,10.421264,10.937446,9.225843,15.341799,10.244828,7.812613,NaN,19.969118,9.0,11.254935


In [375]:
def weighted_rows(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

weighted_rows(0)

,Year,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,2015.2,1633.063333,8.780301,NaN,8.99573,NaN,7.328551,NaN,NaN,11.024737,NaN,10.172424,8.251429,7.659412,13.555,9.661707,14.53,10.095512,NaN,10.003452,NaN,NaN,10.981282,15.516207,10.631149,10.083604,11.922472,12.511188,13.77931,NaN,NaN,26.414804,8.914127,11.354317,1555.42,8.711647,NaN,8.683708,NaN,7.393451,NaN,NaN,11.781579,NaN,9.404545,7.766964,8.706536,13.047222,8.562331,15.483333,9.979538,NaN,9.366931,NaN,NaN,10.797657,18.777586,10.421264,10.937446,9.225843,15.341799,10.244828,7.812613,NaN,19.969118,9.0,11.254935


In [376]:
weighted_rows(4)

,Year,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
4,2014.5,1299.601667,7.427711,NaN,7.497191,NaN,6.050725,NaN,NaN,10.444737,NaN,8.025758,6.271429,5.808824,10.019444,6.937805,12.216667,7.986799,NaN,7.294881,NaN,NaN,7.744872,11.708621,8.995977,8.815584,8.326966,10.840329,10.072414,7.362162,NaN,20.412255,8.130952,9.092376,1177.281667,6.735542,NaN,6.95412,NaN,5.396618,NaN,NaN,7.644737,NaN,7.25,5.892857,5.126471,9.125,5.984321,14.4,7.154455,NaN,6.749286,NaN,NaN,6.792735,9.577586,7.520115,7.061688,NaN,9.366255,NaN,NaN,NaN,21.968137,7.078571,8.348444


####  Row Prediction

In [377]:
pb1 = pb.filter(like='to')

In [378]:
pb1.shape

(152, 31)

In [379]:
weighted_pb = pd.DataFrame()

In [380]:
weighted_pb

""


In [381]:
pb1.shape[0]

152

In [382]:
for i in range(pb1.shape[0]):
    weighted_pb = weighted_pb.append(pace_rec(i), ignore_index=True)

In [383]:
weighted_pb

,Year,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,2015.200000,1633.063333,8.780301,NaN,8.995730,NaN,7.328551,NaN,NaN,11.024737,NaN,10.172424,8.251429,7.659412,13.555000,9.661707,14.530000,10.095512,NaN,10.003452,NaN,NaN,10.981282,15.516207,10.631149,10.083604,11.922472,12.511188,13.779310,NaN,NaN,26.414804,8.914127,11.354317,1555.420000,8.711647,NaN,8.683708,NaN,7.393451,NaN,NaN,11.781579,NaN,9.404545,7.766964,8.706536,13.047222,8.562331,15.483333,9.979538,NaN,9.366931,NaN,NaN,10.797657,18.777586,10.421264,10.937446,9.225843,15.341799,10.244828,7.812613,NaN,19.969118,9.000000,11.254935
1,2015.900000,1600.908333,8.530462,NaN,8.642060,NaN,7.136739,NaN,NaN,10.579474,NaN,10.323333,8.197619,9.731176,12.670556,9.763415,26.012222,10.437855,NaN,9.588333,NaN,NaN,11.878889,15.162414,10.990000,10.245310,11.248315,12.037037,11.796552,NaN,NaN,25.973824,8.691587,12.109681,1517.773333,8.354596,NaN,10.127603,NaN,6.990918,NaN,NaN,10.181053,NaN,9.656970,7.692143,7.214379,12.987778,8.410840,15.360000,9.520891,NaN,8.833929,NaN,NaN,9.572460,17.994483,10.120920,10.172727,10.742697,14.720370,12.400000,7.970270,NaN,18.449706,8.649683,10.802511
2,2015.600000,1624.785000,8.934056,NaN,9.205581,NaN,7.498986,NaN,NaN,11.043684,NaN,10.303333,8.095000,8.087647,13.553333,9.348049,14.681111,10.395578,NaN,9.934405,NaN,NaN,10.987094,15.418276,10.395287,9.981447,10.761423,11.851852,12.256322,NaN,NaN,24.708235,8.877143,11.245023,1550.673333,8.424779,NaN,8.512584,NaN,7.241908,NaN,NaN,12.332632,NaN,9.743636,7.602381,7.596078,12.963333,8.484553,15.368889,9.751122,NaN,9.344709,NaN,NaN,10.761909,15.673103,10.316782,10.755844,9.225843,15.162434,10.244828,8.398198,NaN,20.489314,9.747460,11.058064
3,2015.400000,1624.103333,8.752490,NaN,8.966142,NaN,7.320580,NaN,NaN,11.148421,NaN,10.070909,8.158571,7.721176,13.507778,9.504390,14.668889,10.200792,NaN,10.035000,NaN,NaN,10.885983,15.497241,10.566207,10.568182,11.248315,12.569444,14.210345,NaN,NaN,25.796667,8.859365,11.329602,1529.871667,8.525703,NaN,8.463109,NaN,6.968331,NaN,NaN,11.302632,NaN,8.986364,7.688095,7.562745,12.600000,8.383469,15.316667,9.519967,NaN,9.05992

In [384]:
weighted_pb = comb.join(weighted_pb)

In [385]:
weighted_pb

,category,gender,Year,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
0,Masters Men 1 (40-49),Male,2015.200000,1633.063333,8.780301,NaN,8.995730,NaN,7.328551,NaN,NaN,11.024737,NaN,10.172424,8.251429,7.659412,13.555000,9.661707,14.530000,10.095512,NaN,10.003452,NaN,NaN,10.981282,15.516207,10.631149,10.083604,11.922472,12.511188,13.779310,NaN,NaN,26.414804,8.914127,11.354317,1555.420000,8.711647,NaN,8.683708,NaN,7.393451,NaN,NaN,11.781579,NaN,9.404545,7.766964,8.706536,13.047222,8.562331,15.483333,9.979538,NaN,9.366931,NaN,NaN,10.797657,18.777586,10.421264,10.937446,9.225843,15.341799,10.244828,7.812613,NaN,19.969118,9.000000,11.254935
1,Masters Men 1 (40-49),Male,2015.900000,1600.908333,8.530462,NaN,8.642060,NaN,7.136739,NaN,NaN,10.579474,NaN,10.323333,8.197619,9.731176,12.670556,9.763415,26.012222,10.437855,NaN,9.588333,NaN,NaN,11.878889,15.162414,10.990000,10.245310,11.248315,12.037037,11.796552,NaN,NaN,25.973824,8.691587,12.109681,1517.773333,8.354596,NaN,10.127603,NaN,6.990918,NaN,NaN,10.181053,NaN,9.656970,7.692143,7.214379,12.987778,8.410840,15.360000,9.520891,NaN,8.833929,NaN,NaN,9.572460,17.994483,10.120920,10.172727,10.742697,14.720370,12.400000,7.970270,NaN,18.449706,8.649683,10.802511
2,Masters Men 2 (50-59),Male,2015.600000,1624.785000,8.934056,NaN,9.205581,NaN,7.498986,NaN,NaN,11.043684,NaN,10.303333,8.095000,8.087647,13.553333,9.348049,14.681111,10.395578,NaN,9.934405,NaN,NaN,10.987094,15.418276,10.395287,9.981447,10.761423,11.851852,12.256322,NaN,NaN,24.708235,8.877143,11.245023,1550.673333,8.424779,NaN,8.512584,NaN,7.241908,NaN,NaN,12.332632,NaN,9.743636,7.602381,7.596078,12.963333,8.484553,15.368889,9.751122,NaN,9.344709,NaN,NaN,10.761909,15.673103,10.316782,10.755844,9.225843,15.162434,10.244828,8.398198,NaN,20.489314,9.747460,11.058064
3,Masters Men 2 (50-59),Male,2015.400000,1624.103333,8.752490,NaN,8.966142,NaN,7.320580,NaN,NaN,11.148421,NaN,10.070909,8.158571,7.721176,13.507778,9.504390,14.668889,10.200792,NaN,10.035000,NaN,NaN,10.885983,15.497241,10.566207,10.568182,11.248315,12.569444,14.210345,NaN,NaN,25.796667,8.859365,11.329602,1529.871667,8.525703,

In [386]:
# Read in Split Timings
weighted_pb.to_csv('../../data/wea_ws_weighted_pb.csv', encoding = "ISO-8859-1")

In [387]:
# Read in Split Timings
w_pb = pd.read_csv('../../data/wea_ws_weighted_pb.csv', encoding = "ISO-8859-1")

In [388]:
w_pb.iloc[[8]]

,Unnamed: 0,category,gender,Year,time,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,AVG_Pace,time_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,AVG_Pace_pb
8,8,Senior Women (23-39),Female,2014.5,1227.611667,7.249478,NaN,7.185243,NaN,5.93591,NaN,NaN,9.085263,NaN,7.525455,5.926429,5.674118,9.549444,6.736098,11.491111,7.623729,NaN,7.037024,NaN,NaN,7.244957,10.911034,8.255862,7.857143,7.765169,10.25463,9.641379,6.767568,NaN,15.076569,7.791111,8.303243,1161.49,6.633434,NaN,6.79176,NaN,5.33901,NaN,NaN,7.463158,NaN,7.421212,5.835714,5.105882,9.036111,6.090592,13.983333,7.132508,NaN,6.708333,NaN,NaN,6.8,9.524138,7.478161,7.137662,NaN,9.442387,NaN,NaN,NaN,21.77598,6.976984,8.282594


####  means

In [389]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(10)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


7.13317861722893

In [390]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


7.134699458467442

In [391]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


6.925441244938826

In [392]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


11.00611987647752

##  Best pace row

In [393]:
def best_pace_rec(index):
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.sort_values(by='AVG_Pace_pb', ascending=True)
    row_pace = row_pace.head(1)
   # row_pace = pd.DataFrame(row_pace)
   # row_pace = row_pace.T
    row_pace = row_pace.filter(like='to')
    row_pace.index = query.index

    return row_pace


best_pace_rec(5)

,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
5,6.987952,NaN,7.640449,NaN,5.434783,NaN,NaN,9.473684,NaN,9.090909,6.607143,5.764706,10.138889,6.463415,11.777778,7.821782,NaN,8.071429,NaN,NaN,8.717949,12.413793,9.655172,9.87013,NaN,10.925926,NaN,NaN,NaN,24.656863,7.18254,6.307229,NaN,6.466292,NaN,5.07971,NaN,NaN,7.5,NaN,7.378788,5.928571,4.911765,8.527778,5.963415,9.722222,6.727723,NaN,6.714286,NaN,NaN,6.252137,8.810345,7.316092,6.642857,NaN,8.388889,NaN,NaN,NaN,20.088235,6.507937


In [394]:
pb_top = pd.DataFrame()

In [395]:
pb_top

""


In [396]:
for i in range(pb1.shape[0]):
    pb_top = pb_top.append(best_pace_rec(i), ignore_index=True)

In [397]:
pb_top

,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
0,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.812651,NaN,8.782022,NaN,7.001449,NaN,NaN,9.794737,NaN,7.475758,7.221429,6.723529,11.011111,7.339024,12.344444,7.749835,NaN,8.022619,NaN,NaN,7.677778,10.934483,8.635632,NaN,8.439326,NaN,9.555172,NaN,NaN,16.370588,7.217460
1,8.552209,NaN,8.333333,NaN,6.729469,NaN,NaN,10.000000,NaN,8.939394,NaN,11.294118,11.666667,NaN,26.000000,8.509901,NaN,7.860714,NaN,NaN,9.658120,13.793103,9.885057,7.922078,NaN,10.370370,NaN,NaN,NaN,27.352941,10.230159,7.210843,NaN,7.477528,NaN,5.876812,NaN,NaN,8.552632,NaN,8.136364,6.821429,5.382353,9.777778,6.207317,11.055556,7.618812,NaN,6.428571,NaN,NaN,7.363248,11.224138,8.120690,7.811688,NaN,10.240741,NaN,NaN,NaN,23.617647,7.809524
2,7.612450,NaN,8.273408,NaN,6.666667,NaN,NaN,9.368421,NaN,9.696970,7.321429,9.058824,13.472222,7.804878,12.666667,8.316832,NaN,7.785714,NaN,NaN,8.717949,12.413793,10.000000,8.701299,NaN,10.925926,NaN,NaN,NaN,25.220588,9.484127,7.669880,NaN,8.514607,NaN,7.084058,NaN,NaN,9.357895,NaN,8.684848,6.092857,6.317647,10.338889,6.297561,11.533333,7.823762,NaN,7.128571,NaN,NaN,7.892308,14.337931,8.475862,8.771429,NaN,10.385185,NaN,6.686486,NaN,12.136275,10.842857
3,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.812651,NaN,8.782022,NaN,7.001449,NaN,NaN,9.794737,NaN,7.475758,7.221429,6.723529,11.011111,7.339024,12.344444,7.749835,NaN,8.022619,NaN,NaN,7.677778,10.934483,8.635632,NaN,8.439326,NaN,9.555172,NaN,NaN,16.370588,7.217460
4,6.987952,NaN,7.640449,NaN,5.434783,NaN,NaN,9.473684,NaN,9.090909,6.607143,5.764706,10.138889,6.463415,11.777778,7.821782,NaN,8.071429,NaN,NaN,8.717949,12.413793,9.655172,9.870130,NaN,10.925926,NaN,NaN,NaN,24.656863,7.182540,6.307229,NaN,6.466292,NaN,5.079710,NaN,NaN,7.500000,NaN,7.37878

In [398]:
pb_top = comb.join(pb_top)

In [399]:
pb_top

,category,gender,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
0,Masters Men 1 (40-49),Male,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.812651,NaN,8.782022,NaN,7.001449,NaN,NaN,9.794737,NaN,7.475758,7.221429,6.723529,11.011111,7.339024,12.344444,7.749835,NaN,8.022619,NaN,NaN,7.677778,10.934483,8.635632,NaN,8.439326,NaN,9.555172,NaN,NaN,16.370588,7.217460
1,Masters Men 1 (40-49),Male,8.552209,NaN,8.333333,NaN,6.729469,NaN,NaN,10.000000,NaN,8.939394,NaN,11.294118,11.666667,NaN,26.000000,8.509901,NaN,7.860714,NaN,NaN,9.658120,13.793103,9.885057,7.922078,NaN,10.370370,NaN,NaN,NaN,27.352941,10.230159,7.210843,NaN,7.477528,NaN,5.876812,NaN,NaN,8.552632,NaN,8.136364,6.821429,5.382353,9.777778,6.207317,11.055556,7.618812,NaN,6.428571,NaN,NaN,7.363248,11.224138,8.120690,7.811688,NaN,10.240741,NaN,NaN,NaN,23.617647,7.809524
2,Masters Men 2 (50-59),Male,7.612450,NaN,8.273408,NaN,6.666667,NaN,NaN,9.368421,NaN,9.696970,7.321429,9.058824,13.472222,7.804878,12.666667,8.316832,NaN,7.785714,NaN,NaN,8.717949,12.413793,10.000000,8.701299,NaN,10.925926,NaN,NaN,NaN,25.220588,9.484127,7.669880,NaN,8.514607,NaN,7.084058,NaN,NaN,9.357895,NaN,8.684848,6.092857,6.317647,10.338889,6.297561,11.533333,7.823762,NaN,7.128571,NaN,NaN,7.892308,14.337931,8.475862,8.771429,NaN,10.385185,NaN,6.686486,NaN,12.136275,10.842857
3,Masters Men 2 (50-59),Male,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.812651,NaN,8.782022,NaN,7.001449,NaN,NaN,9.794737,NaN,7.475758,7.221429,6.723529,11.011111,7.339024,12.344444,7.749835,NaN,8.022619,NaN,NaN,7.677778,10.934483,8.635632,NaN,8.439326,NaN,9.555172,NaN,NaN,16.370588,7.217460
4,Senior Women (23-39),Female,6.987952,NaN,7.640449,NaN,5.434783,NaN,NaN,9.473684,NaN,9.090909,6.607143,5.764706,10.138889,6.463415,11.777778,7.821782,NaN,8.071429,N

In [400]:
# Read in Split Timings
pb_top.to_csv('../../data/wea_ws_pb_top.csv', encoding = "ISO-8859-1")

In [401]:
# Read in Split Timings
pb_top = pd.read_csv('../../data/wea_ws_pb_top.csv', encoding = "ISO-8859-1")

In [402]:
pb_top.iloc[[8]]

,Unnamed: 0,category,gender,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
8,8,Senior Women (23-39),Female,7.06747,NaN,7.166292,NaN,5.77971,NaN,NaN,8.2,NaN,7.927273,5.914286,5.729412,9.505556,6.419512,10.866667,7.526733,NaN,7.342857,NaN,NaN,7.464957,10.2,8.475862,8.122078,NaN,12.051852,NaN,4.794595,NaN,9.562745,7.668254,6.307229,NaN,6.466292,NaN,5.07971,NaN,NaN,7.5,NaN,7.378788,5.928571,4.911765,8.527778,5.963415,9.722222,6.727723,NaN,6.714286,NaN,NaN,6.252137,8.810345,7.316092,6.642857,NaN,8.388889,NaN,NaN,NaN,20.088235,6.507937


## Get Best

In [403]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(5)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


7.6439089314167745

In [404]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


9.7407447823920474

# Calculate Predictions

In [405]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [406]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,11.804720,10.505747,9.740745,11.254935,11.253518,10.769839
1,12.242579,9.251986,9.547084,10.802511,10.789664,10.484158
2,10.913974,13.759780,9.450123,11.058064,11.058902,11.051297
3,11.844286,11.503364,9.773393,11.124727,11.124374,11.236092
4,9.080386,9.710111,8.560169,8.348444,8.345238,8.338878
5,9.954235,9.710111,8.488027,8.471251,8.471576,8.470431
6,9.483453,8.170402,8.414558,8.452512,8.447986,8.448995
7,8.623698,8.170402,8.610858,8.436564,8.436982,8.438380
8,7.989676,8.170402,7.977780,8.282594,8.283825,8.283172
9,7.524734,7.846284,7.071498,7.515874,7.516087,7.524777


In [407]:
df.iloc[[66]]

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
66,8.964305,9.699724,8.576359,8.394363,8.390438,8.382607


Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [408]:
comb.join(df)

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Masters Men 1 (40-49),Male,11.804720,10.505747,9.740745,11.254935,11.253518,10.769839
1,Masters Men 1 (40-49),Male,12.242579,9.251986,9.547084,10.802511,10.789664,10.484158
2,Masters Men 2 (50-59),Male,10.913974,13.759780,9.450123,11.058064,11.058902,11.051297
3,Masters Men 2 (50-59),Male,11.844286,11.503364,9.773393,11.124727,11.124374,11.236092
4,Senior Women (23-39),Female,9.080386,9.710111,8.560169,8.348444,8.345238,8.338878
5,Senior Women (23-39),Female,9.954235,9.710111,8.488027,8.471251,8.471576,8.470431
6,Senior Women (23-39),Female,9.483453,8.170402,8.414558,8.452512,8.447986,8.448995
7,Senior Women (23-39),Female,8.623698,8.170402,8.610858,8.436564,8.436982,8.438380
8,Senior Women (23-39),Female,7.989676,8.170402,7.977780,8.282594,8.283825,8.283172
9,Senior Men (23-39),Male,7.524734,7.846284,7.071498,7.515874,7.516087,7.524777


In [409]:
df = comb.join(df)

In [410]:
df.to_csv('../../data/wea_ws_pace_predictions.csv')